In [2]:
import pandas as pd
import numpy as np
#from tqdm.notebook import tqdm
import scipy.sparse as sp
import pickle

In [3]:
npz_file = np.load('./tmp/data.npz', allow_pickle=True)
train_ids = npz_file["train_ids"]
valid_ids = npz_file["valid_ids"]
NUM_ITEMS = 43038

In [4]:
len(train_ids) == len(valid_ids)

True

In [5]:
def make_coo_row(item_ids, num_items=NUM_ITEMS):
    idx = item_ids
    values = np.ones(len(item_ids)).astype(np.float32)

    return sp.coo_matrix(
        (values, ([0] * len(idx), idx)), 
        shape=(1, num_items),
    )

In [27]:
values = np.ones(len(idx)).astype(np.float32)
sp.coo_matrix(values, ([0] * len(idx), idx))

<1x184 sparse matrix of type '<class 'numpy.float32'>'
	with 184 stored elements in COOrdinate format>

In [8]:
rows = []
for user_history in train_ids:
    rows.append(make_coo_row(user_history))

In [11]:
X_sparse = sp.vstack(rows).tocsr()

In [12]:
pickle.dump(X_sparse, open('./tmp/X_sparse.pickled', 'wb'))

# i2i

**item-to-item (item2item, i2i)** - один из типов коллаборативной фильтрации, основанный на близости между товарами.

Основная идея: для каждого товара строим список наиболее похожих на него товаров. 

<center><img src="imgs/ub-ib-cf.jpg" width=700></center>

### Прогнозирование рейтинга
* Посчитаем сходство между товарами $s \in \mathbb{R}^{I \times I}$
* Для товара $i$ найти оцененные пользователем $u$ похожие товары: $N(i)$

$$ \hat{R}_{ui} = \frac{\sum_{j \in N(i)} s_{ij}R_{uj}}{\sum_{j \in N(i)} \left| s_{ij}\right|} $$


**Ядро алгоритма - матрица схожести $s$**

(Какого размера эта матрица? Как можно определять "похожесть"?)

i2i рекомендации в implicit - это сохранение заранее просчитанных списков "похожих" для каждого айтема.

Количество "похожих" регулируется параметром K

<center><img src='imgs/item_demo.gif'></center>

In [16]:
#!pip install implicit
#import implicit

In [17]:
#import implicit
from src.metrics import normalized_average_precision

In [ ]:
model = implicit.nearest_neighbours.CosineRecommender(K=200)
model.fit(X_sparse.T)

In [ ]:
model.recommend(10, X_sparse, N=10, filter_already_liked_items=False)

In [ ]:
m_ap = []
for i, gt_ids in tqdm(enumerate(valid_ids)):
    rec_raw = model.recommend(i, X_sparse, N=30, filter_already_liked_items=False)
    rec_ids = [x[0] for x in rec_raw]
    m_ap.append(normalized_average_precision(gt_ids, rec_ids, k=30))
print(np.mean(m_ap))

# Эффект количества соседей

In [ ]:
for k in [1, 10, 100]:
    _model = implicit.nearest_neighbours.CosineRecommender(K=k)
    _model.fit(X_sparse.T)
    _model.save('./tmp/model_{}'.format(k))

In [ ]:
! ls -lah ./tmp/model_*

# Эффекты скорости

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=30, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=300, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=3000, filter_already_liked_items=False)

In [ ]:
%%timeit
_ = model.recommend(10, X_sparse, N=30000, filter_already_liked_items=False)

In [ ]:
%%timeit
selected_items = list(range(10))
_ = model.rank_items(10, X_sparse, selected_items)

In [ ]:
# Почему есть разница?

# Эффект размера рекомендаций

In [ ]:
preds = model.recommend(10, X_sparse, N=30000, filter_already_liked_items=False)
preds 

In [ ]:
len(preds)

In [ ]:
X_sparse[10].nnz 

In [ ]:
# Сколько минимум может быть рекомендаций? А максимум?

In [ ]:
selected_items = list(range(10))
model.rank_items(10, X_sparse, selected_items)

In [ ]:
for k in [1, 10, 30, 100, 300]:
    _model = implicit.nearest_neighbours.CosineRecommender(K=k)
    _model.fit(X_sparse.T)
    preds = _model.recommend(10, X_sparse, N=30000, filter_already_liked_items=False)
    print('k: {}, num_preds: {}'.format(k, len(preds)))

Какие есть места для твиков?
- в алгоритме
- в препроцессинге
- в постановке задачи

( _вопрос со звездочкой_ ) Что мы оптимизируем этим алгоритмом? / Какая оптимизационная задача решается? <br>
( _вопрос со звездочкой_ ) А что будет, если матрицу **X_sparse** перевернуть/транспонировать?